## Picking stocks with a Quantitative Momentum strategy 

- simple yet useful method to optimize the process of chossing stocks

The steps involved in selecting quantitative momentum stocks are:
- Importing the required packages
- Extracting the list of all nifty50 stocks symbols
- Pulling Intraday data of all the stocks in the nifty50
- calculating percentage change and momentum of all stocks
- Finding stocks with greater momentum
- Backtesting with a equal-weight portfolio

## step 1: Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import requests
from math import floor
from scipy.stats import percentileofscore as score
from statistics import mean

In [ ]:
import time

In [ ]:
import yfinance as yf

In [ ]:
#pd.set_option('display.max_rows',None)
#pd.set_option('display.max_columns',None)

## step 2: Extracting the list of all nifty50 stocks symbols

In [ ]:
nifty = pd.read_csv('ind_nifty50list.csv')

In [ ]:
nifty_symbol = nifty['Symbol'].to_list()

In [ ]:
nifty_symbol

In [ ]:
yf_symbol = []
for count,symbol in enumerate(nifty_symbol):
    stock_symbol = symbol+'.NS'
    print(count,symbol,end='\r')
    yf_symbol.append(stock_symbol)
    time.sleep(0.5)

In [ ]:
yf_symbol

In [ ]:
df = yf.download(yf_symbol,start='2021-06-04',end='2021-06-05',interval='5m')['Close']

In [ ]:
df

In [ ]:
close_price = df[-2:-1].T

In [ ]:
close_price.reset_index(inplace=True)

In [ ]:
type(close_price)

In [ ]:
#close_price.rename(columns = {'2021-06-04 15:20:00+05:30:'Close'},inplace=True)
close_price.columns = ['Symbol','Close']

In [ ]:
close_price

## step 4: calculating percentage change and momentum

## percentage change

In [ ]:
pc = []
vol = []
var = []
for i in df.columns:
    var.append(df[i].var())
    vol.append(df[i].std())
    pc.append(df[i].pct_change().sum())

In [ ]:
nifty_momentum = pd.DataFrame(columns=['symbol','day_change','var'])
nifty_momentum['symbol'] = df.columns
nifty_momentum['day_change'] = pc
nifty_momentum['vol'] = vol
nifty_momentum['var'] = var

In [ ]:
df.pct_change().sum()

In [ ]:
nifty_momentum

In [ ]:
nifty_momentum[nifty_momentum.vol == nifty_momentum.vol.max()]

In [ ]:
nifty_momentum.sort_values(by='vol',ascending=False)

In [ ]:
nifty_momentum['day_change'].var()

In [ ]:
np.sqrt(nifty_momentum['day_change'].var())

In [ ]:
nifty_momentum['day_change'].std()

## calculating momentum

In [ ]:
nifty_momentum['momentum'] = 'N/A'
for i in range(len(nifty_momentum)):
    nifty_momentum.loc[i,'momentum'] = score(nifty_momentum.day_change,nifty_momentum.loc[i,'day_change'])/100
nifty_momentum['momentum'] = nifty_momentum['momentum'].astype(float)

In [ ]:
nifty_momentum.dtypes

In [ ]:
nifty_momentum

In [ ]:
top_picks = nifty_momentum.nlargest(10,'momentum')
top_picks

In [ ]:
nifty_momentum.sort_values('day_change',ascending=False).head(10)

In [ ]:
df2 = pd.read_csv('ind_nifty50list.csv')

In [ ]:
df2['Symbol'] = df2['Symbol']+'.NS'

In [ ]:
df2

In [ ]:
df3 = pd.merge(nifty_momentum,df2,how='inner',left_on='symbol',right_on='Symbol')

In [ ]:
df3

In [ ]:
df3.columns

In [ ]:
df4 = df3[['symbol', 'day_change', 'momentum','vol', 'Company Name', 'Industry']]

In [ ]:
df4

In [ ]:
df5 = pd.merge(nifty_momentum,close_price,how='inner',left_on='symbol',right_on='Symbol')

In [ ]:
df5

In [ ]:
df3[['symbol','day_change']]